In [2]:
from t_smiles.DataSet.STDTokens import CTokens, STDTokens_Frag_File
from t_smiles.MolUtils.RDKUtils.Frag.RDKFragUtil import Fragment_Alg

from t_smiles.DataSet.Graph.CNJMolAssembler import CNJMolAssembler

from t_smiles.DataSet.Graph.CNJMolUtil import CNJMolUtil
from t_smiles.DataSet.Graph.CNJTMol import CNJMolUtils

# from DataSet.Graph.CNJTMol import preprocess TODO: unused

Test encoding algorithm

In [11]:
smls = "CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F"  # celecoxib

dec_algs = [
    Fragment_Alg.Vanilla,
    Fragment_Alg.JTVAE,
    Fragment_Alg.BRICS,
    Fragment_Alg.BRICS_Base,
    Fragment_Alg.MMPA,
    Fragment_Alg.Scaffold,
    Fragment_Alg.BRICS_DY,
    Fragment_Alg.MMPA_DY,
    Fragment_Alg.Scaffold_DY,
    # Fragment_Alg.RBrics_DY,
]

print("[smls is]:", smls)

ctoken = CTokens(STDTokens_Frag_File(None), max_length=256, invalid=True, onehot=False)

encoder = CNJMolUtils()

for dec_alg in dec_algs:
    combine_sml, combine_smt, amt_bfs_smarts = encoder.encode_single(
        smls, ctoken, dec_alg
    )

    print("[dec_alg is]:", dec_alg.name)
    print("[TSSA/TSDY]:", combine_sml)
    print("[TSID     ]:", combine_smt)

[smls is]: CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F
[dec_alg is]: Vanilla
[TSSA/TSDY]: CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&
[TSID     ]: CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&
[dec_alg is]: JTVAE
[TSSA/TSDY]: CC&C1=CC=CC=C1&CC&C1=CNN=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&
[TSID     ]: CC&C1=CC=CC=C1&CC&C1=CNN=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&
Mol could not be Bricsed: CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F
[dec_alg is]: BRICS
[TSSA/TSDY]: CC&C1=CC=CC=C1&CC&C1=CNN=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&
[TSID     ]: CC&C1=CC=CC=C1&CC&C1=CNN=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&
[dec_alg is]: BRICS_Base
[TSSA/TSDY]: CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&
[TSID     ]: CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&
[dec_alg is]: MMPA
[TSSA/TSDY]: CC&C1=CC=CC=C1&CC&C1=CNN=C1&CN&C1=CC=CC=C1^CC^CS&C&NS(=O)=O&CF&&&&

Test decoding Algorithm

In [8]:
maxlen = 512
vocab_file = r"../RawData/Chembl/Test/Chembl_test.smi.[MMPA_DY][237]_token.voc.smi"

ctoken = CTokens(
    STDTokens_Frag_File(vocab_file),
    is_pad=True,
    pad_symbol=" ",
    startend=True,
    max_length=maxlen,
    flip=False,
    invalid=True,
    onehot=False,
)

# SMILES	 = 'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F' #Celecoxib

bfs_ex_list = [
    "CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F&&&",  # TS_Vanilla
    "CC&C1=CC=CC=C1&CC&C1=C[NH]N=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&",  # TSSA-J
    "CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&",  # TSSA-B
    "CC&C1=CC=CC=C1&CC&C1=C[NH]N=C1&CN&C1=CC=CC=C1^CC^CS&C&N[SH](=O)=O&CF&&&&FCF&&",  # TSSA-M
    "CC&C1=CC=C(C2=CC=NN2C2=CC=CC=C2)C=C1&CC&FC(F)F^CS&&N[SH](=O)=O&&&",  # TSSA-S
    "CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&",  # TSDY-B
    "*C&*C1=CC=C(*)C=C1&*C1=CC(*)=NN1*&*C(*)(F)F&*F^*C1=CC=C(*)C=C1&&*S(N)(=O)=O&&&",  # TSDY-M
    "*C&*C1=CC=C(C2=CC(*)=NN2C2=CC=C(*)C=C2)C=C1&*C(F)(F)F&&*S(N)(=O)=O&&",  # TSDY-S
    "CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&",  # TSID-B
    "[1*]C&[1*]C1=CC=C([2*])C=C1&[2*]C1=CC([3*])=NN1[5*]&[3*]C([4*])(F)F&[4*]F^[5*]C1=CC=C([6*])C=C1&&[6*]S(N)(=O)=O&&&",  # TSID_M
    "[1*]C&[1*]C1=CC=C(C2=CC([2*])=NN2C2=CC=C([3*])C=C2)C=C1&[2*]C(F)(F)F&&[3*]S(N)(=O)=O&&",  # TSID-S
]
asm_list = ["CALG_TSSA", "CALG_TSDY", "CALG_TSID"]


mol_util = CNJMolUtil()


assembler = CNJMolAssembler()

num_samples = 1

for bfs in bfs_ex_list:
    bfs_ex = "".join(bfs.strip().split(" "))
    bfs_ex_smiles = mol_util.split_ex_smiles(bfs_ex, delimiter="^")
    for alg in asm_list:
        re_smils = assembler.decode_single(
            bfs_ex, ctoken, alg, n_samples=1, p_mean=None
        )
        print(
            f"Input: {bfs_ex}\nbfs_ex_smiles: {bfs_ex_smiles}\nAlg: {alg}\nDecoded_smile: {re_smils}\n\n"
        )

'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F'
Input: CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F&&&
bfs_ex_smiles: ['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F', '&', '&', '&']
Alg: CALG_TSSA
Decoded_smile: ('Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2)cc1', ['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F', '&', '&', '&'], [['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F']], 0)


'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F'
Input: CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F&&&
bfs_ex_smiles: ['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F', '&', '&', '&']
Alg: CALG_TSDY
Decoded_smile: ('Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2)cc1', ['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F', '&', '&', '&'], [['CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F']], 0)


'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F'
Input: CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S